# I część

## 1 zad
Należy wykonać strojenie modelu BERT-base na danych pozyskanych w ramach
Zadania 1. Można również wykorzystać inne dane treningowo-testowe. Proszę
wykorzystać model polskojęzyczny (np. allegro/herbert-base-cased).
Uwaga: proszę zastosować również adapter PEFT!!! [1,5pkt]
1. Wykonać strojenie dla zadania klasyfikacji tekstu
(BertForSequenceClassification)
1. Wykonać strojenie dla zadania klasyfikacji tokenów
(BertForTokenClassification)
1. Wydzielić niewielki zbiór testowy i ocenić jakość predykcji.

In [1]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AutoModelForTokenClassification

Load datasets

In [2]:
# load dataset
from datasets import load_dataset, DatasetDict, concatenate_datasets 
polemo = load_dataset("clarin-pl/polemo2-official")
kpwr = load_dataset("clarin-pl/kpwr-ner")

In [3]:
polemo_n_cls = len(polemo['train'].features['target'].names)
kpwr_n_cls = len(kpwr['train'].features['ner'].feature.names)

In [4]:
# add validation part to train
polemo = DatasetDict(
    {
        "train": concatenate_datasets([polemo['train'], polemo['validation']]),
        "test": polemo['test']
    }
)

Load models

In [41]:
tokenizer = AutoTokenizer.from_pretrained("allegro/herbert-base-cased")
model_seq = AutoModelForSequenceClassification.from_pretrained("allegro/herbert-base-cased", num_labels=polemo_n_cls)
model_token = AutoModelForTokenClassification.from_pretrained("allegro/herbert-base-cased", num_labels=kpwr_n_cls)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at allegro/herbert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of BertForTokenClassification were not initialized from the model checkpoint at allegro/herbert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Load PEFT adapter

In [45]:
from peft import LoraConfig, get_peft_model

config_seq = LoraConfig(
    task_type="SEQ_CLS",
)

config_token = LoraConfig(
    task_type="TOKEN_CLS",
)

model_seq_peft = get_peft_model(model_seq, config_seq)
model_token_peft = get_peft_model(model_token, config_token)

Tokenize data

In [11]:
def preprocess_text_classification(examples):
    return tokenizer(examples['text'], padding="max_length", truncation=True, max_length=128)

data_seq = polemo.map(preprocess_text_classification, batched=True)
data_seq = data_seq.rename_column("target", "labels")
data_seq.set_format("torch", columns=["input_ids", "attention_mask", "labels"])

def preprocess_token_classification(examples):
    tokenized_inputs = tokenizer(examples['tokens'], padding="max_length", truncation=True, max_length=128, is_split_into_words=True)
    labels = []
    for i, label in enumerate(examples["ner"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        label_ids = [-100 if word_id is None else label[word_id] for word_id in word_ids]
        labels.append(label_ids)
    tokenized_inputs["labels"] = labels
    return tokenized_inputs

data_token = kpwr.map(preprocess_token_classification, batched=True)
data_token.set_format("torch", columns=["input_ids", "attention_mask", "labels"])

Map:   0%|          | 0/7396 [00:00<?, ? examples/s]

Fine-tune models

In [88]:
from transformers import TrainingArguments, Trainer

training_args_seq = TrainingArguments(
    output_dir="./results/seq",
    num_train_epochs=10,
    per_device_train_batch_size=64,
    per_device_eval_batch_size=64,
    weight_decay=0.01,
    logging_dir="./logs/seq",
    save_total_limit=1,
    evaluation_strategy="epoch"
)

training_args_token = TrainingArguments(
    output_dir="./results/token",
    num_train_epochs=10,
    per_device_train_batch_size=64,
    per_device_eval_batch_size=64,
    weight_decay=0.01,
    logging_dir="./logs/token",
    save_total_limit=1,
    evaluation_strategy="epoch"
)

In [91]:
trainer_seq = Trainer(
    model=model_seq_peft,
    args=training_args_seq,
    train_dataset=data_seq["train"]
)

trainer_seq.train()

Step,Training Loss
500,0.925500
1000,0.502000


TrainOutput(global_step=1160, training_loss=0.6803898646913725, metrics={'train_runtime': 233.8942, 'train_samples_per_second': 316.211, 'train_steps_per_second': 4.96, 'total_flos': 4881936873799680.0, 'train_loss': 0.6803898646913725, 'epoch': 10.0})

In [92]:
trainer_token = Trainer(
    model=model_token_peft,
    args=training_args_token,
    train_dataset=data_token["train"]
)

trainer_token.train()

Step,Training Loss
500,1.624600
1000,0.558500
1500,0.456400
2000,0.418500


TrainOutput(global_step=2190, training_loss=0.7329265646738549, metrics={'train_runtime': 435.8267, 'train_samples_per_second': 320.288, 'train_steps_per_second': 5.025, 'total_flos': 9176600680473600.0, 'train_loss': 0.7329265646738549, 'epoch': 10.0})

Test model performance

In [129]:
import evaluate
import numpy as np

accuracy = evaluate.load("accuracy")

def compute_metrics(pred, target):
    pred = np.argmax(pred, axis=1)
    return accuracy.compute(predictions=pred, references=target)

In [ ]:
pred_seq = trainer_seq.predict(data_seq["test"])

In [133]:
acc_seq = compute_metrics(pred_seq.predictions, pred_seq.label_ids)['accuracy']
acc_seq

0.8390243902439024

In [ ]:
pred_token = trainer_token.predict(data_token["test"])

In [144]:
acc_token = compute_metrics(pred_token.predictions.reshape(-1, kpwr_n_cls), pred_token.label_ids.flatten())['accuracy']
acc_token

0.1509838364561647

## 2 zad
Zwizualizować przestrzeń wektorową dla przykładów testowych (wystarczy
jedynie klasyikacja tekstu) i zrobić interaktywny wykres przedstawiający to, w
jaki sposób przypadki testowe organizują się w przestrzeni wektorowej
względem przypisanych etykiet. [1pkt]

- W przypadku wykorzystania adaptera PEFT proszę wykorzystać reprezentacje
tokenu [CLS] jako reprezentację tekstu.

In [5]:
from peft import AutoPeftModelForSequenceClassification, AutoPeftModelForTokenClassification

In [21]:
model_seq = AutoPeftModelForSequenceClassification.from_pretrained("results/seq/checkpoint-1160", num_labels=polemo_n_cls)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at allegro/herbert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [164]:
import pandas as pd

df_seq = pd.DataFrame(columns=["text", "label", "cls_embed"])

In [153]:
samples_seq = polemo.map(preprocess_text_classification, batched=True)
samples_seq = samples_seq.rename_column("target", "labels")
samples_seq.set_format("torch", columns=["input_ids", "attention_mask", "labels"])
samples = samples_seq["test"]
texts = samples_seq["test"]["text"]

Extract embeddings

In [157]:
import torch

cls_tokens = []
for i in range(0, len(samples), 64):
    batch = samples[i:i+64]
    with torch.no_grad():
        output = model_seq(**batch, output_hidden_states=True)
    cls_embed = output.hidden_states[-1][:, 0, :].cpu().numpy()
    cls_tokens.append(cls_embed)
    
cls_tokens = np.concatenate(cls_tokens)

In [165]:
df_seq["text"] = texts
df_seq["label"] = samples["labels"]
df_seq["cls_embed"] = cls_tokens.tolist()

In [167]:
df_seq['label_txt'] = df_seq['label'].apply(lambda x: polemo['train'].features['target'].int2str(x))
df_seq['text_br'] = df_seq['text'].apply(lambda x: ' '.join([x + '<br>' if (idx % 10 == 0 and idx != 0) else x for idx, x in enumerate(x.split(' '))]))

In [80]:
from sklearn.manifold import TSNE
import plotly.express as px

Visualization

In [170]:
tsne = TSNE(n_components=2, random_state=42, perplexity=50, max_iter=1000)
X_tsne = tsne.fit_transform(cls_tokens)

df_seq["x-tsne"] = X_tsne[:, 0]
df_seq["y-tsne"] = X_tsne[:, 1]


# Create interactive scatter plot
fig = px.scatter(
    df_seq, x='x-tsne', y='y-tsne', color='label_txt',
    hover_data={'text_br': True, 'label_txt': True},
    title='Sentences t-SNE Visualization', width=1200, height=1200
)

# Show plot
fig.show()

## 3 zad
Powtórzyć podpunkty 1 i 2 z wykorzystaniem klas GPT2ForSequenceClassification
i GPT2ForTokenClassification. Proszę wykorzystać model polskojęzyczny (np.
sdadas/polish-gpt2-medium) Uwaga: proszę zastosować również adapter
PEFT!!! [1pkt]
- W przypadku wykorzystania adaptera PEFT proszę wykorzystać reprezentacje
ostatniego tokenu sekwencji tekstowej jako reprezentację tekstu.
- Proszę również zamaskować tokeny paddingu i znacznik <eos> za pomocą
etykietki -100.

## 4 zad
Wykorzystać token [MASK] w modelu BERT do powiększenia zbiorów
treningowych (ang. data augmentation). [0,5pkt]

## 5 zad
Wykorzystać generatywne zdolności GPT-2 i różne hiperparametry generacji, np.
temperatura, top-p, top-k, powiększenia zbiorów treningowych (ang. data
augmentation). [1pkt]
- Uwaga! W podpunktach 4 i 5 można również połączyć wykorzystanie modelu
BERT i GPT-2 w ramach zagadnienia powiększania zbiorów danych. Wystarczy
wytrenować jeden model na powiększonym zbiorze danych.

# II część - Analiza własności modeli językowych (4 pkt)

## 1 zad
Należy wykorzystać dołączony do zadania zbiór danych z podpunktu A i
odtworzyć badanie z podpunktu B dla modeli BERT i GPT-2 . [2pkt]
1. wykonać analizę anizotropii (Anisotropy)
1. wykonać analizę zależności od kontekstu (Context-Specificity)

## 2 zad
Proszę wykorzystać technikę Parameter Projection omówioną w artykule z
podpunktu C niniejszej instrukcji, następnie dokonać analizy modeli BERT-base i
GPT-2 [1,5pkt]
1. do analizy należy wybrać modele polskojęzyczny
sdadas/polish-gpt2-medium,
1. proszę utworzyć listy z załączników C.1 oraz C.2 przedstawionych w
artykule,
1. należy przeanalizować utworzone listy i podsumować uzyskane rezultaty.